In [ ]:
import brainpy as bp
import brainpy.math as bm

bp.math.enable_x64()
bp.math.set_platform('cpu')

gamma = 0.641  # Saturation factor for gating variable
tau = 0.1  # Synaptic time constant [sec]
a = 270.  #  Hz/nA
b = 108.  # Hz
d = 0.154  # sec

I0 = 0.3255  # background current [nA]
JE = 0.2609  # self-coupling strength [nA]
JI_opst = -0.0497  # cross-coupling strength to the opposite direction [nA]
JI_othg = -0.0497  # cross-coupling strength to the othorgnal direction [nA]
JAext = 0.00052  # Stimulus input strength [nA]
Ib = 0.3255  # The background input


# Define the model

the synapic current activation function is as below
$$
r_i = F(I_i)=\frac{aI_i-b}{1-exp(-d(aI_i-b))}
\tag{1}
$$

The synapic drive is defined as below
$$
\frac{dS_1}{dt}=F(I_1)\gamma(1-S_1)-S_1/\tau _s \\
$$
$$ 
\frac{dS_2}{dt}=F(I_2)\gamma(1-S_2)-S_2/\tau _s
$$
$$ 
\frac{dS_3}{dt}=F(I_3)\gamma(1-S_3)-S_3/\tau _s
$$
$$ 
\frac{dS_4}{dt}=F(I_4)\gamma(1-S_4)-S_4/\tau _s
$$ 


The net current into each population
directions 1/2/3/4 indicate 0/90/180/270 deg
$$
I_1 = J_E S_1 + J_{Iothg} S_2 + J_{Iopst} S_3 + J_{Iothg} S_4 + I_b + J_{ext}\mu_1
$$

$$
I_2 = J_E S_2 + J_{Iothg} S_1 + J_{Iopst} S_4 + J_{Iothg} S_3 + I_b + J_{ext}\mu_2
$$
$$
I_3 = J_E S_3 + J_{Iothg} S_2 + J_{Iopst} S_1 + J_{Iothg} S_4 + I_b + J_{ext}\mu_3
$$
$$
I_4 = J_E S_3 + J_{Iothg} S_1 + J_{Iopst} S_2 + J_{Iothg} S_3 + I_b + J_{ext}\mu_4
$$

The input current
we need to think about this, use the tuning curve of motion
$$
u = const * u_0 * exp(-\frac{(dir - \theta_k)^2}{\sigma^2})
$$
here, $\sigma$ is the bandwidth of direction selective tuning curve 


In [ ]:
# implement the model
# note that we should modify the mu part
@bp.odeint
def int_s1(s1, t, s2, s3, s4, coh=0.5, mu=20.):
    I1 = JE * s1 + JI_othg * s2 + JI_opst * s3 + JI_othg * s4 + Ib + JAext * mu * (1. + coh)
    r1 = (a * I1 - b) / (1. - bm.exp(-d * (a * I1 - b)))
    return - s1 / tau + (1. - s1) * gamma * r1

@bp.odeint
def int_s2(s2, t, s1, s3, s4 coh=0.5, mu=20.):
    I2 = JE * s2 + JI_othg * s1 + JI_opst * s4 + JI_othg * s3 + Ib + JAext * mu * (1. + coh)
    r2 = (a * I2 - b) / (1. - bm.exp(-d * (a * I2 - b)))
    return - s2 / tau + (1. - s2) * gamma * r2

@bp.odeint
def int_s3(s3, t, s1, s2, s4, coh=0.5, mu=20.):
    I3 = JE * s3 + JI_othg * s2 + JI_opst * s1 + JI_othg * s4 + Ib + JAext * mu * (1. + coh)
    r3 = (a * I3 - b) / (1. - bm.exp(-d * (a * I3 - b)))
    return - s3 / tau + (1. - s3) * gamma * r3

@bp.odeint
def int_s4(s4, t, s1, s2, s3, coh=0.5, mu=20.):
    I4 = JE * s4 + JI_othg * s1 + JI_opst * s2 + JI_othg * s3 + Ib + JAext * mu * (1. + coh)
    r4 = (a * I4 - b) / (1. - bm.exp(-d * (a * I4 - b)))
    return - s4 / tau + (1. - s4) * gamma * r4

1. Define visualization function
2. run brainpy runner